```{contents}
```
## **Subgraph Node in LangGraph**

A **Subgraph Node** is a node whose execution logic is itself a complete **LangGraph graph**.
It enables **hierarchical composition of workflows**, allowing large systems to be built from smaller, reusable graphs.

---

### **1. Motivation: Why Subgraphs Exist**

Large LLM systems are **structurally complex**:

* Multi-agent pipelines
* Reusable reasoning modules
* Nested workflows
* Domain-specific task engines

Without subgraphs, graphs become:

* Monolithic
* Hard to maintain
* Impossible to test in isolation

A **Subgraph Node** solves this by making graphs **composable**.

---

### **2. Conceptual Model**

```
Main Graph
  |
  |--- Node A
  |
  |--- Subgraph Node ──▶ (Graph B)
  |                       ├── Node B1
  |                       ├── Node B2
  |                       └── Node B3
  |
  |--- Node C
```

From the main graph’s perspective, the **entire subgraph behaves as one node**.

---

### **3. Formal Definition**

A **Subgraph Node**:

* Receives the **parent state**
* Executes its own internal graph
* Returns an updated state to the parent graph

---

### **4. Creating a Subgraph**

#### **Step 1 — Build the Subgraph**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class SubState(TypedDict):
    text: str
    sentiment: str

def analyze(state):
    return {"sentiment": "positive"}

sub_builder = StateGraph(SubState)
sub_builder.add_node("analyze", analyze)
sub_builder.set_entry_point("analyze")
sub_builder.add_edge("analyze", END)

sentiment_graph = sub_builder.compile()
```

---

#### **Step 2 — Embed Subgraph as Node**

```python
class MainState(TypedDict):
    text: str
    sentiment: str

builder = StateGraph(MainState)
builder.add_node("sentiment_node", sentiment_graph)
builder.set_entry_point("sentiment_node")
builder.add_edge("sentiment_node", END)

graph = builder.compile()
```

Invocation:

```python
graph.invoke({"text": "LangGraph is excellent"})
```

---

### **5. State Mapping Between Graphs**

| Direction         | Description                 |
| ----------------- | --------------------------- |
| Parent → Subgraph | Fields required by subgraph |
| Subgraph → Parent | Fields produced by subgraph |

Subgraph only reads/writes its declared schema.
The parent graph receives merged updates.

---

### **6. Why Subgraphs Are Critical in Production**

| Benefit              | Explanation                       |
| -------------------- | --------------------------------- |
| Modularity           | Isolated, reusable logic          |
| Scalability          | Build large systems cleanly       |
| Testing              | Unit-test subgraphs independently |
| Parallel development | Teams own different subgraphs     |
| Maintainability      | Localized changes                 |
| Security             | Limit state exposure              |

---

### **7. Common Subgraph Design Patterns**

| Pattern              | Purpose                    |
| -------------------- | -------------------------- |
| Planner Subgraph     | Planning logic             |
| Agent Swarm Subgraph | Multi-agent coordination   |
| Retrieval Subgraph   | Knowledge grounding        |
| Validation Subgraph  | Compliance & safety        |
| Toolchain Subgraph   | Complex tool orchestration |
| Reflection Subgraph  | Self-correction loops      |

---

### **8. Advanced Variant — Cyclic Subgraph**

A subgraph can contain its own **loops**:

```
Main Graph
   |
   └── Subgraph
          ↺ Reason → Act → Observe
```

This creates nested autonomous behaviors.

---

### **9. Debugging & Observability**

Each subgraph:

* Has its own trace
* Has its own checkpoints
* Appears as a single node in parent visualization

---

### **10. Mental Model**

> **A Subgraph Node is a microservice for reasoning.**

It encapsulates:

* Control flow
* State evolution
* Agent behavior

into a single reusable computational unit.

### Demonstration

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

# ---------- Subgraph State ----------
class SentimentState(TypedDict):
    text: str
    sentiment: str

# ---------- Subgraph Logic ----------
def analyze_sentiment(state: SentimentState):
    text = state["text"].lower()
    if "good" in text:
        return {"sentiment": "positive"}
    if "bad" in text:
        return {"sentiment": "negative"}
    return {"sentiment": "neutral"}

# ---------- Build Subgraph ----------
sub_builder = StateGraph(SentimentState)
sub_builder.add_node("analyze", analyze_sentiment)
sub_builder.set_entry_point("analyze")
sub_builder.add_edge("analyze", END)

sentiment_graph = sub_builder.compile()

In [2]:
# ---------- Main Graph State ----------
class MainState(TypedDict):
    text: str
    sentiment: str
    decision: str

def decide(state: MainState):
    s = state["sentiment"]
    if s == "positive":
        return {"decision": "promote"}
    if s == "negative":
        return {"decision": "escalate"}
    return {"decision": "review"}

# ---------- Build Main Graph ----------
builder = StateGraph(MainState)

# Subgraph is added exactly like a normal node
builder.add_node("sentiment_node", sentiment_graph)
builder.add_node("decision_node", decide)

builder.set_entry_point("sentiment_node")
builder.add_edge("sentiment_node", "decision_node")
builder.add_edge("decision_node", END)

graph = builder.compile()


In [3]:
result = graph.invoke({"text": "This product is very good"})
print(result)


{'text': 'This product is very good', 'sentiment': 'positive', 'decision': 'promote'}
